# Fine-tuning RadBert #
Full parameter fine tuning of RadBERT model (RadBERT-RoBERTa-4m model from the RadBERT paper). Fine tuning carried out on the multi-label muti-class classification of reports, where each report can have multiple labels (For ex, a report can havel label consolidation-right and consolidation-2)

In [1]:
import os
import time, datetime
import codecs
from itertools import product

import random
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

/home/users/pranav.rao/miniconda3/envs/TutorialCuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Seeding everything for reproducibility
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [3]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 1000)

torch.set_printoptions(linewidth=200)

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
device='cuda:0'

## Forward pass ##
Implementing RadBERTMultiClassMulti Label PyTorch Model

In [6]:
class RadBERTMultiClassMultiLabel(nn.Module):
    """
    RadBERTMultiClassMultiLabel: Model expects batches of natural language sentences, will
    classify reports with multiple label
    """
    def __init__(self, num_classes, checkpoint, device):
        super().__init__()
        self.num_classes = num_classes
        self.checkpoint = checkpoint
        self.device = device

        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint)
        self.transformer_encoder = AutoModel.from_pretrained(self.checkpoint)
        self.transformer_encoder_hidden_size = self.transformer_encoder.config.hidden_size
        self.linear_classifier = nn.Linear(self.transformer_encoder_hidden_size, self.num_classes)
    
    def forward(self, x):
        tokenized_inp = self.tokenizer(x, padding=True, truncation=True, return_tensors='pt').to(self.device)
        encoder_out = self.transformer_encoder(**tokenized_inp)
        logits = self.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits


In [7]:
checkpoint = 'UCSD-VA-health/RadBERT-RoBERTa-4m'
#labels_subset = "normal tuberculosis opacity bronchialdilation density parenchymalopacity ett aorticenlargement mediastinalwidening mediastinalmass\
#        copd prominentbronchovascularmarkings bronchitis markings vascularprominence interval interstitiallungdisease bluntedcp effusion cardiomegaly\
#        consolidation subtle_normal peffusion lineandtube thickening haziness hilarprominence hilar inhomogenousopacity rotation\
#        calcification unfoldedaorta bandlikeopacity aorticcalcification aorticknucklecalcification fibrosis suture cardiacshift degenspine nodule\
#        pneumonia inspiration fracture pneumonitis justfibrosis lesion nonaorticcalcification tuberculosispure pleuralthickening feedingtube".split()
labels_subset = [e.strip() for e in open('/home/users/pranav.rao/MiniTasks/Radbert/labels_top50_withloc.txt', 'r').readlines()]
num_classes = len(labels_subset)

In [ ]:
radbert_multi_model = RadBERTMultiClassMultiLabel(num_classes, checkpoint, device).to(device)

In [ ]:
print(radbert_multi_model)
print(list(map(lambda x : x.shape, radbert_multi_model.parameters())))

## Custom Loss function ##
Custom loss function for multi class multi label classification to handle uncertain tags (tags have value 0 -> absent, 1 -> present, -100 -> uncertain)

In [80]:
class MultiClassMultiLabel(nn.Module):
    def __init__(self, uncertain_label, device):
        super(MultiClassMultiLabel, self).__init__()
        self.uncertain_label = uncertain_label
        self.device = device
    
    def forward(self, output, target, penalize_certainity=False):
        certain_mask = (target != self.uncertain_label)
        loss_func = nn.MultiLabelSoftMarginLoss(weight=certain_mask.type(torch.float))
        if not penalize_certainity:
            return loss_func(output, target)
        else:
            certainity_loss_func = nn.MultiLabelSoftMarginLoss(weight=(~certain_mask).type(torch.float))
            ones = torch.ones(output.shape, device=self.device)
            zeros = torch.zeros(output.shape, device=self.device)
            return loss_func(output, target) + 0.1 * (certainity_loss_func(output, ones) + certainity_loss_func(output, zeros))

In [81]:
multiclass_multilabel_loss = MultiClassMultiLabel(-100, device).to(device)

### Testing MultiClassMultiLabel loss function ###

In [82]:
logit_tensor = torch.Tensor([[-1.0, 2.0, 1.0, 5.0, -3.0], [4.0, -2.0, 1.0, -1.0, 2.5]]).to(device)
target_tensor_act = torch.Tensor([[0, 1, -100, 0, 0], [1, -100, 0, 0, 1]]).to(device)

In [83]:
#certain_mask = (target_tensor_act != -100)
#print(certain_mask.type(torch.float))
#loss_func = nn.MultiLabelSoftMarginLoss(weight=(certain_mask).type(torch.float))
#print(loss_func(logit_tensor, target_tensor_act))
print(multiclass_multilabel_loss(logit_tensor, target_tensor_act))
print(multiclass_multilabel_loss(logit_tensor, target_tensor_act, penalize_certainity=True))

tensor(0.7219, device='cuda:0')
tensor(0.7607, device='cuda:0')


In [84]:
import math
from math import log, exp

def log_sigmoid(x):
    return log(1/(1+exp(-1*x)))

In [85]:
loss1 = -1 * (1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.0) + 0.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(-5.0) + 1.0 * log_sigmoid(3.0)) / 5.0
loss2 = -1 * (1.0 * log_sigmoid(4.0) + 0.0 * log_sigmoid(2.0) + 1.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.5)) / 5.0

print(loss1)
print(loss2)
print((loss1 + loss2)/2.0)

1.0990984797248111
0.34471260744936094
0.721905543587086


In [86]:
loss1 = -1 * (1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.0) + 0.1 * (log_sigmoid(-1.0) + log_sigmoid(1.0)) + 1.0 * log_sigmoid(-5.0) + 1.0 * log_sigmoid(3.0)) / 5.0
loss2 = -1 * (1.0 * log_sigmoid(4.0) + 0.1 * (log_sigmoid(2.0) + log_sigmoid(-2.0)) + 1.0 * log_sigmoid(-1.0) + 1.0 * log_sigmoid(1.0) + 1.0 * log_sigmoid(2.5)) / 5.0

print(loss1)
print(loss2)
print((loss1 + loss2)/2.0)

1.13162894722554
0.3897897278910799
0.76070933755831


## Custom DataLoader and Dataset ##
Read csv file, get the report from path and prepare the data

In [87]:
class ReportTagsDataset(Dataset):
    def __init__(self, tags_csv_file, report_base_path, labels_subset=None, text_transform=None, target_transform=None):
        self.report_base_path = report_base_path
        self.tags_csv_file = tags_csv_file

        self.tags_df = pd.read_csv(self.tags_csv_file)
        self.column_names = list(self.tags_df.columns.values)
        self.column_names[0] = 'filename'
        self.tags_df.columns = self.column_names

        self.labels_subset = labels_subset
        self.text_transform = text_transform
        self.target_transform = target_transform
    
    def __len__(self):
        return self.tags_df.shape[0]
    
    def __getitem__(self, index):
        report_path = os.path.join(self.report_base_path, self.tags_df.iloc[index, 0].split('/')[-1] + '.txt')
        #report_text = open(report_path).read()
        report_text = codecs.open(report_path, 'r', encoding='utf-8', errors='ignore').read()
        if self.labels_subset is None:
            target_list = torch.Tensor(list(self.tags_df.iloc[index][1:]))
        else:
            target_list = torch.Tensor(list(self.tags_df[self.labels_subset].iloc[index]))
        return report_text, target_list

In [88]:
#report_base_path = "/models_common_e2e/cxr_data/reports/training"
train_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/train'
test_reports_base_path = '/home/users/pranav.rao/MiniTasks/Radbert/data/test'
train_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_train.csv'
test_tags_file = '/home/users/pranav.rao/Downloads/report_tags_25k_test.csv'

In [89]:
train_data = ReportTagsDataset(train_tags_file, train_reports_base_path, labels_subset=labels_subset)
test_data = ReportTagsDataset(test_tags_file, test_reports_base_path, labels_subset=labels_subset)

In [90]:
#train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
#test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

### Testing Dataset and DataLoader ###

In [91]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))

In [92]:
print(train_features)
print(train_labels)
print(f"Feature batch shape: {len(train_features)}")
print(f"Labels batch shape: {len(train_labels)}")
report_text = train_features[0]
print(report_text)
label = train_labels[0]
print(f"Label: {label}")

('6016667|909308|Investigation: X-Ray Chest AP Supine (Portable)\nResults:\nArtifact +.\nSubtle haze seen in left mid and lower zones.  \nCardiac size and pulmonary vasculature cannot be assessed (AP view).\nVisualised bones appear normal.\nAdvise: Clinical correlation. \n6016667|909308|Investigation: X-Ray Chest AP Supine (Portable)\nResults:\nArtifact +.\nSubtle haze seen in left mid and lower zones.  \nCardiac size and pulmonary vasculature cannot be assessed (AP view).\nVisualised bones appear normal.\nAdvise: Clinical correlation. ', '6229228|4281617|X-Ray Chest PA/AP View of 22-FEB-2018:\nResults:\nNo focal lesion seen in the lung parenchyma. \nCP angles and domes of the diaphragm are normal.\nBoth hila are normal. \nCardiac size and configuration is normal.\nTrachea is central; no mediastinal shift is seen.\nBony thorax and soft tissues of the chest wall are normal.\n6229228|4281617|X-Ray Chest PA/AP View of 22-FEB-2018:\nResults:\nNo focal lesion seen in the lung parenchyma. \n

In [93]:
with torch.no_grad():
    logit_tensor = radbert_multi_model(train_features)

In [94]:
print(logit_tensor)
print(logit_tensor.shape)

tensor([[-2.8738, -0.3806, -0.2766,  ..., -3.9745, -4.2900, -2.7723],
        [ 0.6204, -3.2360, -2.9979,  ..., -4.6585, -4.4890, -3.9174],
        [-3.9546, -0.1608,  0.2215,  ..., -3.7216, -3.9332, -2.7011],
        ...,
        [-3.4354, -0.9878, -0.4057,  ..., -4.1158, -4.3209, -3.2361],
        [ 1.0332, -3.2712, -3.2869,  ..., -4.5339, -4.3962, -3.9428],
        [ 0.8829, -3.2148, -3.2063,  ..., -4.5850, -4.4622, -3.9426]], device='cuda:0')
torch.Size([32, 76])


## Fine Tuning ##
Fine tuning the RadBERT model for tags prediction task

### Adam Optimizer ###
Using Adam optimizer with learning rate 3e-5, beta1 = 0.9, beta2 = 0.99, l2 weight decay of 0.01

In [ ]:
lr = 3e-5
beta1 = 0.9
beta2 = 0.99
l2_weight_decay = 0.01

In [ ]:
optimizer = torch.optim.Adam(radbert_multi_model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=l2_weight_decay)

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = radbert_multi_model(inputs)

        # Compute the loss and its gradients
        loss = multiclass_multilabel_loss(outputs, labels)
        loss.backward()
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 50 == 49:
            last_loss = running_loss / 50 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_dataloader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

### The full fine-tuning loog ###

In [ ]:
total_epochs = 5

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))

for epoch in range(total_epochs):
    # Make sure gradient tracking is on, and do a pass over the data
    print('EPOCH {}:'.format(epoch + 1))
    radbert_multi_model.train(True)
    avg_loss = train_one_epoch(epoch, writer)
    model_path = '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_{}_{}'.format(timestamp, epoch)
    torch.save(radbert_multi_model.state_dict(), model_path)

    # Set the model to evaluation mode, disabling dropout and using population, statistics for batch normalization
    running_vloss = 0.0
    radbert_multi_model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_dataloader):
            vinputs, vlabels = vdata
            vlabels = vlabels.to(device)
            voutputs = radbert_multi_model(vinputs)
            vloss = multiclass_multilabel_loss(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch + 1)
    writer.flush()

## Analyzing fine-tuned models ##

In [8]:
sentence_list = ["The report shows small right-sided pleural effusion", "The report shows small left-sided pleural effusion",\
    "The report shows large right-sided pleural effusion", "The report shows large left-sided pleural effusion",\
    "There are no abnormalities in the report",\
    "There is severe consolidation in the left side","There is severe consolidation in the right side",\
    "There is mild consolidation in the right side", "There is mild consolidation in the left side"
]

sentence1_base = "A <SizeModifier> <AbnormalReport> can be seen in the report in the <LocationModifier> part"
sentence2_base = "The report shows a <SizeModifier> <LocationModifier> <AbnormalReport>"
size_modifiers = ['small', 'large']
loc_modifiers = ['upper-left', 'lower-left', 'right-sided', 'left-sided']
abnormal_report = ['pleural effusion']

l1 = [sentence1_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]
l2 = [sentence2_base.replace('<SizeModifier>', size_mod).replace('<LocationModifier>', loc_mod).replace('<AbnormalReport>', ab_rep) for size_mod, loc_mod, ab_rep in product(size_modifiers, loc_modifiers, abnormal_report)]

negative_sentences = ['The report shows no pleural effusion', 'The report shows no consolidation on any side']
all_sentence_list = l1 + l2 + negative_sentences + sentence_list[4:]

In [9]:
print('\n'.join(all_sentence_list))

A small pleural effusion can be seen in the report in the upper-left part
A small pleural effusion can be seen in the report in the lower-left part
A small pleural effusion can be seen in the report in the right-sided part
A small pleural effusion can be seen in the report in the left-sided part
A large pleural effusion can be seen in the report in the upper-left part
A large pleural effusion can be seen in the report in the lower-left part
A large pleural effusion can be seen in the report in the right-sided part
A large pleural effusion can be seen in the report in the left-sided part
The report shows a small upper-left pleural effusion
The report shows a small lower-left pleural effusion
The report shows a small right-sided pleural effusion
The report shows a small left-sided pleural effusion
The report shows a large upper-left pleural effusion
The report shows a large lower-left pleural effusion
The report shows a large right-sided pleural effusion
The report shows a large left-sid

In [10]:
def get_sentence_embeddings(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt').to(device)
        encoder_out = model.transformer_encoder(**tokenized_inp)
    return encoder_out.last_hidden_state[:, 0, :]

In [11]:
def calc_cosine_sim_matrix(sentence_embeddings):
    cosine_sim_matrix = F.cosine_similarity(sentence_embeddings.unsqueeze(1), sentence_embeddings.unsqueeze(0), dim=2)
    return cosine_sim_matrix.cpu().detach().numpy()

In [67]:
model_path = '/home/users/pranav.rao/MiniTasks/Radbert/ModelPool/model_20230915_162326_1'
radbert_multi_model = RadBERTMultiClassMultiLabel(num_classes, checkpoint, device).to(device)
radbert_multi_model.load_state_dict(torch.load(model_path))

Some weights of RobertaModel were not initialized from the model checkpoint at UCSD-VA-health/RadBERT-RoBERTa-4m and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [68]:
all_embeddings = get_sentence_embeddings(radbert_multi_model, all_sentence_list)

In [69]:
print(all_embeddings)
print(all_embeddings.shape)

tensor([[-0.1358, -0.2679, -0.1015,  ..., -1.6266, -0.1136,  0.1293],
        [-0.1420, -0.2995, -0.0863,  ..., -1.6932, -0.0958,  0.1089],
        [-0.1208, -0.2779, -0.0425,  ..., -1.5986, -0.1241,  0.0354],
        ...,
        [-0.0948, -0.3249, -0.1927,  ..., -1.4449,  0.0059, -0.1009],
        [-0.1498, -0.3140, -0.2099,  ..., -1.4756, -0.0100, -0.1531],
        [-0.1275, -0.3363, -0.1981,  ..., -1.3927,  0.0045, -0.1569]], device='cuda:0')
torch.Size([23, 768])


In [70]:
cosine_sim = calc_cosine_sim_matrix(all_embeddings)
pd.DataFrame(cosine_sim)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.000000,0.998954,0.995846,0.996561,0.996421,0.996448,0.991896,0.992975,0.976323,0.974620,0.965111,0.965426,0.971744,0.971747,0.960403,0.961362,0.843338,0.683170,0.504187,0.937940,0.935778,0.943563,0.945827
1,0.998954,1.000000,0.995790,0.996455,0.994314,0.996445,0.990734,0.991796,0.974386,0.975110,0.963961,0.964162,0.968810,0.971160,0.958136,0.959060,0.844715,0.686206,0.507842,0.935199,0.933183,0.942278,0.944318
2,0.995846,0.995790,1.000000,0.999123,0.992671,0.993484,0.996470,0.995806,0.979112,0.978800,0.975555,0.974326,0.974918,0.976108,0.970934,0.970376,0.847906,0.686642,0.502932,0.941440,0.941281,0.948388,0.948884
3,0.996561,0.996455,0.999123,1.000000,0.993060,0.993873,0.995202,0.996408,0.979806,0.979434,0.975046,0.975378,0.975194,0.976382,0.970162,0.971056,0.848259,0.686612,0.503859,0.940905,0.938399,0.946081,0.948725
4,0.996421,0.994314,0.992671,0.993060,1.000000,0.998914,0.995767,0.996582,0.976429,0.973326,0.965516,0.965708,0.977814,0.976380,0.966209,0.967194,0.837255,0.676485,0.497822,0.947539,0.945325,0.946282,0.948490
5,0.996448,0.996445,0.993484,0.993873,0.998914,1.000000,0.995424,0.996285,0.975074,0.974399,0.964705,0.964829,0.975386,0.976348,0.964206,0.965220,0.839508,0.680365,0.501998,0.945141,0.943036,0.945394,0.947401
6,0.991896,0.990734,0.996470,0.995202,0.995767,0.995424,1.000000,0.998991,0.978575,0.976769,0.975369,0.973960,0.980349,0.980009,0.976225,0.975639,0.841320,0.679398,0.496332,0.950600,0.950512,0.950821,0.951155
7,0.992975,0.991796,0.995806,0.996408,0.996582,0.996285,0.998991,1.000000,0.979433,0.977592,0.974854,0.975117,0.980828,0.980520,0.975468,0.976468,0.841652,0.679336,0.497019,0.950371,0.947746,0.948690,0.951307
8,0.976323,0.974386,0.979112,0.979806,0.976429,0.975074,0.978575,0.979433,1.000000,0.998299,0.995355,0.995836,0.997032,0.997011,0.992474,0.993086,0.871684,0.699858,0.508804,0.943540,0.940500,0.942175,0.945735
9,0.974620,0.975110,0.978800,0.979434,0.973326,0.974399,0.976769,0.977592,0.998299,1.000000,0.995032,0.995350,0.993821,0.997075,0.990543,0.991018,0.871339,0.700989,0.509973,0.941469,0.938559,0.941998,0.945366


## Analysing the predictions ##

In [16]:
def get_predictions(model, input):
    with torch.no_grad():
        tokenized_inp = model.tokenizer(input, padding=True, truncation=True, return_tensors='pt').to(device)
        encoder_out = model.transformer_encoder(**tokenized_inp)
        logits = model.linear_classifier(encoder_out.last_hidden_state[:, 0, :])
        return logits.cpu().detach()

In [17]:
def analyse_predictions(prediction_list, sentence_list):
    cols = ['Sentence', 'Best label', 'Best label score', 'Positive label']
    values = list()
    for (report, pred) in zip(sentence_list, prediction_list.numpy()):
        best_label = labels_subset[pred.argmax()]
        best_label_score = pred.max()
        positive_label = ','.join([labels_subset[i] for i, e in enumerate(pred) if e > 0.0])
        values.append([report, best_label, best_label_score, positive_label])
    return pd.DataFrame(values, columns=cols)

In [71]:
predictions = get_predictions(radbert_multi_model, all_sentence_list)

In [72]:
print(predictions)
print(predictions.shape)

tensor([[-1.4666,  0.2100,  0.1021,  ..., -1.3542, -1.9751, -1.0925],
        [-1.4669,  0.2107,  0.1120,  ..., -1.4054, -2.0276, -1.1367],
        [-1.4588,  0.1898,  0.0981,  ..., -1.3341, -1.9431, -1.0768],
        ...,
        [-1.7028,  0.3419,  0.0654,  ..., -1.2263, -1.8088, -0.8866],
        [-1.7040,  0.3233,  0.1575,  ..., -1.2829, -1.8622, -0.9334],
        [-1.6512,  0.3326,  0.1500,  ..., -1.2266, -1.8159, -0.8983]])
torch.Size([23, 76])


In [73]:
analyse_predictions(predictions, all_sentence_list)

,Sentence,Best label,Best label score,Positive label
0,A small pleural effusion can be seen in the report in the upper-left part,opacity,0.209956,"opacity,bronchialdilation,parenchymalopacity,opacity-left,opacity-right"
1,A small pleural effusion can be seen in the report in the lower-left part,opacity,0.210673,"opacity,bronchialdilation,parenchymalopacity,opacity-left"
2,A small pleural effusion can be seen in the report in the right-sided part,opacity,0.189836,"opacity,bronchialdilation,parenchymalopacity,opacity-left"
3,A small pleural effusion can be seen in the report in the left-sided part,opacity,0.176835,"opacity,bronchialdilation,parenchymalopacity,opacity-left"
4,A large pleural effusion can be seen in the report in the upper-left part,opacity,0.277767,"opacity,bronchialdilation,parenchymalopacity,opacity-left,opacity-right"
5,A large pleural effusion can be seen in the report in the lower-left part,opacity,0.280123,"opacity,bronchialdilation,parenchymalopacity,opacity-left,opacity-right"
6,A large pleural effusion can be seen in the report in the right-sided part,opacity,0.256907,"opacity,bronchialdilation,parenchymalopacity,opacity-left,opacity-right"
7,A large pleural effusion can be seen in the report in the left-sided part,opacity,0.247709,"opacity,bronchialdilation,parenchymalopacity,opacity-left,opacity-right"
8,The report shows a small upper-left pleural effusion,opacity-left,0.227936,"opacity,opacity-left,opacity-right"
9,The report shows a small lower-left pleural effusion,opacity-left,0.199550,"opacity,opacity-left"


## Bigger reports ##

In [21]:
dataset1_template = """xr- chest pa  view
findings
lungs: normal.
trachea: normal.
carina: normal.
right and left main bronchi: normal.
pleura: normal.
heart: normal.
right heart border: normal.
left heart border: normal.
pulmonary bay: normal.
pulmonary hila: normal.
aorta: normal.
thoracic spine: normal.
other visualized bones: normal.
visualized soft tissues: normal.
diaphragm: normal.
visualized abdomen:  normal.
visualized neck: normal."""

dataset1_pleural_issue="""
xr- chest pa view
findings
lungs: normal.
trachea: normal.
carina: normal.
right and left main bronchi: normal.
pleura: left costophrenic angle is blunted with thin stripe of homogenous opacity along left lateral chest wall.
heart: normal.
right heart border: normal.
left heart border: normal.
pulmonary bay: normal.
pulmonary hila: normal.
aorta: normal.
thoracic spine: normal.
other visualized bones: normal.
visualized soft tissues: normal.
diaphragm: normal.
visualized abdomen:  normal.
visualized neck: normal."""

dataset2_template="""6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:
results:
post cabg status.
no focal lesion seen in the lung parenchyma.
cp angles and domes of the diaphragm are normal.
both hila are normal. pulmonary vasculature is normal.
cardiac size and configuration is normal.
trachea is central; no mediastinal shift is seen.
bony thorax and soft tissues of the chest wall are normal.
impression: no abnormality detected in the view obtained.
"""

dataset3_template="""
x-ray chest (pa view)
the cardio thoracic ratio is normal.
the heart size and configuration are within normal limits.
the aortic arch is normal.
the lung fields show normal broncho-vascular markings.
both the pulmonary hila are normal in size.
the costophrenic and cardiophrenic recesses and the domes of
diaphragm are normal.
the bones and soft tissues of the chest wall show no abnormality.
impression : normal study.
dr.shakthi kumar
radiologist
ss
________________________________________________________
"""

In [74]:
all_embeddings2 = get_sentence_embeddings(radbert_multi_model, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

In [75]:
print(all_embeddings2)
print(all_embeddings2.shape)

tensor([[ 0.4704,  0.3774,  0.4030,  ..., -1.9102,  0.1381, -0.7943],
        [ 0.2000,  0.1758,  0.1225,  ..., -2.3230,  0.2635, -0.0713],
        [ 0.6248,  0.1511,  0.2548,  ..., -1.6751,  0.2723, -0.9042],
        [ 0.6743, -0.1035,  0.3294,  ..., -2.2018,  0.5157, -0.6146]], device='cuda:0')
torch.Size([4, 768])


In [76]:
cosine_sim2 = calc_cosine_sim_matrix(all_embeddings2)
pd.DataFrame(cosine_sim2)

,0,1,2,3
0,1.000000,0.719094,0.959213,0.914895
1,0.719094,1.000000,0.790105,0.812252
2,0.959213,0.790105,1.000000,0.945881
3,0.914895,0.812252,0.945881,1.000000


In [77]:
predictions2 = get_predictions(radbert_multi_model, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

In [78]:
print(predictions2)
print(predictions2.shape)

tensor([[ 1.2414, -3.3966, -3.3333, -2.6606, -3.5306, -3.1522, -2.2333, -2.8583, -3.0742, -3.4005, -3.1770, -3.1299, -3.0175, -3.2643, -2.7660, -3.4333, -3.4950, -3.6514, -3.2906, -3.8659, -3.0251,
         -3.7838, -3.4601, -3.5937, -3.7952, -3.6711, -3.6230, -3.8643, -3.6509, -3.7307, -3.4599, -4.1361, -3.8187, -4.0084, -4.0785, -3.7621, -3.9141, -3.7104, -4.0454, -4.1266, -3.6017, -4.1304,
         -3.8430, -3.9061, -4.2837, -3.9740, -4.2144, -3.9944, -3.6159, -3.6790, -3.9050, -4.1531, -3.9548, -3.8081, -4.0074, -3.9749, -3.9802, -3.8248, -4.0795, -4.0525, -4.1479, -4.1241, -4.2205,
         -4.0980, -4.1053, -3.8940, -3.6998, -3.6129, -3.9499, -3.6758, -4.0766, -4.1845, -3.8545, -4.3869, -4.1846, -3.8797],
        [-2.1524, -0.8483, -0.7572, -2.4091, -1.5114, -2.5862, -1.7003, -2.4255, -2.3879, -1.4406, -1.7455, -1.6973, -1.3863, -1.7731, -2.3560, -2.3824, -2.0781, -2.3395, -2.2560, -2.6453, -2.7240,
         -2.6160, -2.4378, -2.3860, -2.7411, -2.8063, -2.9282, -2.8632, -2.9122, 

In [79]:
analyse_predictions(predictions2, [dataset1_template, dataset1_pleural_issue, dataset2_template, dataset3_template])

,Sentence,Best label,Best label score,Positive label
0,xr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: normal.\nheart: normal.\nright heart border: normal.\nleft heart bord...,normal,1.241415,normal
1,\nxr- chest pa view\nfindings\nlungs: normal.\ntrachea: normal.\ncarina: normal.\nright and left main bronchi: normal.\npleura: left costophrenic angle is blunted with thin stripe of homogenous op...,bronchialdilation,-0.757183,
2,6191206|3862169|x-ray chest pa/ap view of 09-feb-2018:\nresults:\npost cabg status.\nno focal lesion seen in the lung parenchyma.\ncp angles and domes of the diaphragm are normal.\nboth hila are n...,normal,0.289083,normal
3,\nx-ray chest (pa view)\nthe cardio thoracic ratio is normal.\nthe heart size and configuration are within normal limits.\nthe aortic arch is normal.\nthe lung fields show normal broncho-vascular ...,normal,-0.395444,
